In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import connection

In [3]:
# Read the file
df1 = pd.read_csv('StudentsPerformance.csv')
df1.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
#Changes in dataframe made in connection.py
# df2 = connection.df

columns_names = {
    'gender': 'GENDER',
    'race/ethnicity': 'ETHNICITY',
    'parental level of education': 'PARENTAL EDUCATION LEVEL',
    'lunch': 'LUNCH',
    'test preparation course': 'TEST PREPARATION COURSE',
    'math score': 'MATH SCORE',
    'reading score': 'READING SCORE',
    'writing score': 'WRITING SCORE'
}
    
df1.rename(columns = columns_names, inplace = True)

df1.head()

,GENDER,ETHNICITY,PARENTAL EDUCATION LEVEL,LUNCH,TEST PREPARATION COURSE,MATH SCORE,READING SCORE,WRITING SCORE
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
df1['TEST PREPARATION COURSE'].unique()

array(['none', 'completed'], dtype=object)

### Check Data Quality

In [6]:
#Equal zero, there are no duplicate values
print(len(df1) - len(df1.drop_duplicates()), "\n")

#Counting null values in each column in df with a boolean mask
print(df1.isna().sum(axis = 0), "\n")

for column in df1.columns:
    if column == 'MATH SCORE':
        break
    else:
        print(df1.groupby(column).groups.keys())

print("\n", df1.dtypes, "\n")

for column in df1.loc[:,'MATH SCORE':'WRITING SCORE']:
    print(f"{column}'s column minimum value is:", df1[column].min(), ", and maximum is:", df1[column].max())

(rows, columns) = df1.shape

print("\nAmount of rows are:", rows)
print("Amount of columns are:", columns)


0 

GENDER                      0
ETHNICITY                   0
PARENTAL EDUCATION LEVEL    0
LUNCH                       0
TEST PREPARATION COURSE     0
MATH SCORE                  0
READING SCORE               0
WRITING SCORE               0
dtype: int64 

dict_keys(['female', 'male'])
dict_keys(['group A', 'group B', 'group C', 'group D', 'group E'])
dict_keys(["associate's degree", "bachelor's degree", 'high school', "master's degree", 'some college', 'some high school'])
dict_keys(['free/reduced', 'standard'])
dict_keys(['completed', 'none'])

 GENDER                      object
ETHNICITY                   object
PARENTAL EDUCATION LEVEL    object
LUNCH                       object
TEST PREPARATION COURSE     object
MATH SCORE                   int64
READING SCORE                int64
WRITING SCORE                int64
dtype: object 

MATH SCORE's column minimum value is: 0 , and maximum is: 100
READING SCORE's column minimum value is: 17 , and maximum is: 100
WRITING SCORE's colu

### EDA (Exploratory Data Analysis)

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   GENDER                    1000 non-null   object
 1   ETHNICITY                 1000 non-null   object
 2   PARENTAL EDUCATION LEVEL  1000 non-null   object
 3   LUNCH                     1000 non-null   object
 4   TEST PREPARATION COURSE   1000 non-null   object
 5   MATH SCORE                1000 non-null   int64 
 6   READING SCORE             1000 non-null   int64 
 7   WRITING SCORE             1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [8]:
df1['LUNCH'].value_counts()

LUNCH
standard        645
free/reduced    355
Name: count, dtype: int64

In [9]:
df1.isnull().sum()

GENDER                      0
ETHNICITY                   0
PARENTAL EDUCATION LEVEL    0
LUNCH                       0
TEST PREPARATION COURSE     0
MATH SCORE                  0
READING SCORE               0
WRITING SCORE               0
dtype: int64

In [10]:
df1.describe()

,MATH SCORE,READING SCORE,WRITING SCORE
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [11]:
# Histograma

### Data preparation

In [12]:
from sklearn.preprocessing import OneHotEncoder

X = df1[["TEST PREPARATION COURSE"]]
df1["MEAN_GRADE"] = df1.loc[:, "MATH SCORE":"WRITING SCORE"].mean(axis = 1)
Y = df1[["MEAN_GRADE"]]

# We want a dummy encoding, not One-Hot itself, for that, just to make sure, we set the baseline of which categorie will be set to 0

enc = OneHotEncoder(drop = ["none"]) #Drop the column "none" in the onehotencoding, therefore in the "completed" column "none" is represented as 0
X = enc.fit_transform(X)
print(enc.categories_)
X = X.toarray() #First 10 observations
print(X[0:10])

[array(['completed', 'none'], dtype=object)]
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]]


### Visualizations

### Machine Learning

In [15]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split

import statsmodels.api as sm

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.010, random_state = 2)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape) # Matrices' dimensions

X_train = sm.add_constant(X_train)
model = sm.OLS(Y_train, X_train)
results = model.fit()
print(results.summary())

(990, 1) (10, 1) (990, 1) (10, 1)
                            OLS Regression Results                            
Dep. Variable:             MEAN_GRADE   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     67.21
Date:                Sun, 23 Jul 2023   Prob (F-statistic):           7.53e-16
Time:                        16:30:02   Log-Likelihood:                -4003.6
No. Observations:                 990   AIC:                             8011.
Df Residuals:                     988   BIC:                             8021.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         65.0